### 1. Plots removing outliers and separating sectors

In [ ]:
#### El bueno: y axis en GWh y trend line en MWh

import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from matplotlib.ticker import MaxNLocator, FuncFormatter
import sqlite3 as db
import matplotlib.colors as mcolors

# Connect to your SQLite database
con = db.connect('mydatabase.db')

# Load the table data into a pandas DataFrame
df_grundkapital = pd.read_sql_query('SELECT * FROM "Heat_potential_final_subsectors_F"', con)

# Commit the changes and close the connection
con.commit()
con.close()
# Filter out rows where Capital is 0
df_grundkapital = df_grundkapital[df_grundkapital['Capital'] != 0]

# List of subsectors to analyze with corresponding colormaps
subsectors_colormap = {
    'paper and printing': 'Blues',
    'chemical industry': 'Oranges',
    'iron and steel': 'Greens',
    'non-metallic minerals': 'Purples',
    'refineries': 'Reds',
    'non-ferrous metals': 'Greens'
}

# Set default font sizes for titles, labels, and ticks
plt.rcParams['axes.titlesize'] = 40
plt.rcParams['axes.labelsize'] = 30
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['legend.fontsize'] = 25
plt.rcParams['font.size'] = 25

# Define the directory to save plots
save_dir = 'C:\\Users\\marma\\Documents\\INGENIERIA  INDUSTRIAL\\2º MÁSTER\\TFM\\PLOTS comparing databases'
os.makedirs(save_dir, exist_ok=True)

def lighten_color(color, factor=1.7):
    # Lightens the given color by multiplying the RGB values by a factor.
    color = mcolors.to_rgb(color)
    lightened_color = np.clip(np.array(color) * factor, 0, 1)
    return lightened_color


# Function to remove outliers based on IQR
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
######
# Function to perform plotting with flipped axes and more significant figures, scaling y-axis to GWh
def plot_subsector_with_capital_types(df, subsector_name, base_colormap, title):
    # Remove outliers from both Capital and H2-Potential

    df['level_3_Tj_proportional'] = df['level_1_Tj_proportional'] 
    df = remove_outliers(df, 'Capital')
    df = remove_outliers(df, 'level_3_Tj_proportional')

   # print(subsector_name, df[['level_3_Tj_proportional']])

    # Convert H2-Potential from TWh to GWh
    #df['level_3_Gj_proportional'] = df['level_3_Tj_proportional'] * 1000
    #df['level_3_Mj_proportional'] = df['level_3_Gj_proportional'] * 1000

    # Create the plot
    plt.figure(figsize=(20, 14))

    base_color = plt.get_cmap(base_colormap)(0.6)  # Base color for Stammkapital
    dark_color = darken_colormap(base_colormap)(0.4)  # Darker color for Grundkapital
    light_color = lighten_color(base_color, factor=1.7)  # Lighter color for Hafteinlage

    # Plot Stammkapital, Grundkapital, and Hafteinlage when values are > 0
    if not df[df['Stammkapital'] > 0].empty:
        plt.scatter(df['Stammkapital'], df['level_3_Tj_proportional'], s=100, color=base_color, label='Stammkapital')
    if not df[df['Grundkapital'] > 0].empty:
        plt.scatter(df['Grundkapital'], df['level_3_Tj_proportional'], s=100, color=dark_color, label='Grundkapital')
    if not df[df['Hafteinlage'] > 0].empty:
        plt.scatter(df['Hafteinlage'], df['level_3_Tj_proportional'], s=100, color=light_color, label='Hafteinlage')

    # Extracting the variables
    X = df[['Capital']]
    y = df['level_3_Tj_proportional']
    

    # Linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Predict y values
    y_pred = model.predict(X)
    #y_pred_in_GWh = y_pred / 1000
    
    # Calculate R^2
    r_squared = r2_score(y, y_pred)
    
    # Plot the trend line
    #plt.plot(df['Capital'], y_pred, color='red', linewidth=2)
    plt.plot(df['Capital'], y_pred, color='red', linewidth=2)

    # Add labels and title with padding
    plt.xlabel('Capital (Mio. €)', labelpad=15)  # Add space between x-axis label and axis
    plt.ylabel('Heat-Potential (TJ)', labelpad=15)  # Update y-axis label to GWh
    #plt.title(title, pad=30)  # Add space between the title and the plot

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)

    # Define the custom formatter function for the x-axis (Capital)
    def x_axis_formatter(tick_val, tick_pos):
        return '{:.0f}'.format(tick_val / 1e6)

    # Set the formatter and locator for the x-axis (Capital)
    plt.gca().xaxis.set_major_formatter(FuncFormatter(x_axis_formatter))
    plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

    # Display the equation of the line and R^2 value below the plot with more significant figures
    #equation_text = f'y = {model.coef_[0]:.6f}x + {model.intercept_:.6f}\n$R^2$ = {r_squared:.6f}'
    equation_text = f'y = {model.coef_[0]:.4f}x + {model.intercept_:.4f} (in TJ and Mio. €)\n$R^2$ = {r_squared:.4f}'
    plt.figtext(0.54, -0.05, equation_text, ha="center", fontsize=25, color='red', wrap=True)

    # Add a legend to distinguish between capital types
    plt.legend()

    # Show the plot
    plt.grid(True)
    plt.tight_layout()

    # Save the plot as an image file
    save_path = os.path.join(save_dir, f'Heatpot_{subsector_name.replace(", ", "_").replace(" ", "_")}_flipped_GWh_level1.png')
    plt.savefig(save_path, bbox_inches='tight')  # Ensure everything is within the figure boundary
    plt.show()

# Iterate through each subsector and generate the corresponding plot with flipped axes and y-axis in GWh
for subsector, colormap in subsectors_colormap.items():
    # Filter the DataFrame for the current subsector
    df_subsector = df_grundkapital[df_grundkapital['Eurostat_Name'].str.contains(subsector, case=False, na=False)].copy()
    plot_subsector_with_capital_types(df_subsector, subsector, colormap, f'Heat-Potential in TJ vs. Capital in {subsector.capitalize()}')

### 2. Plots for lower-potential-companies: Threshold to be set as desired

In [ ]:
## Capital en Thsnd.€ y ypred en GWh

import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from matplotlib.ticker import MaxNLocator, FuncFormatter
import sqlite3 as db
import matplotlib.colors as mcolors

# Filter out rows where Capital is 0
df_grundkapital = df_grundkapital[df_grundkapital['Capital'] != 0]

# List of subsectors to analyze with corresponding colormaps
subsectors_colormap = {
    'paper and printing': 'Blues',
    'chemical industry': 'Oranges',
    'iron and steel': 'Greens',
    'non-metallic minerals': 'Purples',
    'refineries': 'Reds',
    'non-ferrous metals': 'Greens'
}


# Set default font sizes for titles, labels, and ticks
plt.rcParams['axes.titlesize'] = 22
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 15
plt.rcParams['font.size'] = 14

# Define the directory to save plots
save_dir = 'C:\\Users\\marma\\Documents\\INGENIERIA  INDUSTRIAL\\2º MÁSTER\\TFM\\PLOTS comparing databases'
os.makedirs(save_dir, exist_ok=True)

# Function to remove outliers based on IQR
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]


# Define the custom formatter function for the x-axis (Capital)
def x_axis_formatter(tick_val, tick_pos):
    return '{:.0f}'.format(tick_val / 1e6)

def darken_color(color, factor=0.5):
    color = mcolors.to_rgb(color)
    darkened_color = np.clip(np.array(color) * factor, 0, 1)
    return darkened_color

def lighten_color(color, factor=1.7):
    color = mcolors.to_rgb(color)
    lightened_color = np.clip(np.array(color) * factor, 0, 1)
    return lightened_color
    
# Function to perform plotting
def plot_low_potentials_for_subsector(df, subsector_name, colormap, threshold=1.2e6):
    # Filter the DataFrame for low H2 potentials

    df_low_h2 = df[df['level_3_Tj_proportional'] <= threshold].copy()

    df_low_h2[['Capital thsnd']] = df_low_h2[['Capital']]*1000
    df_low_h2[['Stammkapital thsnd']] = df_low_h2[['Stammkapital']]*1000
    df_low_h2[['Grundkapital thsnd']] = df_low_h2[['Grundkapital']]*1000
    df_low_h2[['Hafteinlage thsnd']] = df_low_h2[['Hafteinlage']]*1000
    
    df_low_h2 = remove_outliers(df_low_h2, 'Capital thsnd')
    df_low_h2 = remove_outliers(df_low_h2, 'level_3_Tj_proportional')

        
    if df_low_h2.empty:
        print(f"No data available for low H2 potentials in {subsector_name}. Skipping plot.")
        return

    # Create the plot
    plt.figure(figsize=(12, 10))
    
    base_color = plt.get_cmap(colormap)(0.6)  # Base color for Stammkapital
    dark_color = darken_color(base_color, factor=0.7)  # Darker color for Grundkapital
    light_color = lighten_color(base_color, factor=1.3)  # Lighter color for Hafteinlage

    # Plot Stammkapital, Grundkapital, and Hafteinlage when values are > 0
    if not df_low_h2[df_low_h2['Stammkapital'] > 0].empty:
        plt.scatter(df_low_h2[df_low_h2['Stammkapital thsnd'] > 0]['Stammkapital thsnd'], df_low_h2[df_low_h2['Stammkapital thsnd'] > 0]['level_3_Tj_proportional'], s=100, color=base_color, label='Stammkapital')
    if not df_low_h2[df_low_h2['Grundkapital'] > 0].empty:
        plt.scatter(df_low_h2[df_low_h2['Grundkapital thsnd'] > 0]['Grundkapital thsnd'], df_low_h2[df_low_h2['Grundkapital thsnd'] > 0]['level_3_Tj_proportional'], s=100, color=dark_color, label='Grundkapital')
    if not df_low_h2[df_low_h2['Hafteinlage'] > 0].empty:
        plt.scatter(df_low_h2[df_low_h2['Hafteinlage thsnd'] > 0]['Hafteinlage thsnd'], df_low_h2[df_low_h2['Hafteinlage thsnd'] > 0]['level_3_Tj_proportional'], s=100, color=light_color, label='Hafteinlage')


    # Extracting the variables
    X = df_low_h2[['Capital thsnd']]
    y = df_low_h2['level_3_Tj_proportional']

    # Linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Predict y values
    y_pred = model.predict(X)
    # Convert predicted y values back to GWh for plotting
    #y_pred_in_GWh = y_pred / 1000

    # Calculate R^2
    r_squared = r2_score(y, y_pred)

    # Plot the trend line
    plt.plot(df_low_h2['Capital thsnd'], y_pred, color='red', linewidth=2)

    # Add labels and title with padding
    plt.xlabel('Capital [thsnd. €]', labelpad=15)
    plt.ylabel('Heat-Potential [TJ]', labelpad=15)
    #plt.title(f'Low H2 Potentials (<= {threshold} TJ) in {subsector_name.capitalize()}', pad=20)

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)

    # Set the formatter and locator for the x-axis and y-axis
    plt.gca().xaxis.set_major_formatter(FuncFormatter(x_axis_formatter))
    plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
    #plt.gca().yaxis.set_major_formatter(FuncFormatter(y_axis_formatter))
    plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
    

     # Add some padding around the plot edges to avoid data points on the edges
    x_padding = 0.05 * (df_low_h2['Capital thsnd'].max() - df_low_h2['Capital thsnd'].min())
    #y_padding = 0.9 * (df_low_h2['level_3_Tj_proportional'].max() - df_low_h2['level_3_Tj_proportional'].min())

      # Manual padding for y-axis to ensure points don't touch the edges
    y_min, y_max = df_low_h2['level_3_Tj_proportional'].min(), df_low_h2['level_3_Tj_proportional'].max()
    y_padding = (y_max - y_min) * 0.5  # 5% padding
    plt.ylim(y_min - y_padding, y_max + y_padding)
    
    plt.xlim(df_low_h2['Capital thsnd'].min() - x_padding, df_low_h2['Capital thsnd'].max() + x_padding)
    #plt.ylim(df_low_h2['level_3_Tj_proportional'].min() - y_padding, df_low_h2['level_3_Tj_proportional'].max() + y_padding)

    # Adjust the y-axis limits to focus on lower capital values
    if not df_low_h2['Capital thsnd'].empty:
        plt.ylim(df_low_h2['level_3_Tj_proportional'].min() - 0.1, df_low_h2['level_3_Tj_proportional'].max() + 0.1)

    # Display the equation of the line and R^2 value below the plot with more significant figures
    equation_text = f'y = {model.coef_[0]:.4f}x + {model.intercept_:.4f}  (in TJ and Mio. €)\n$R^2$ = {r_squared:.4f}'
    plt.figtext(0.54, -0.05, equation_text, ha="center", fontsize=14, color='red', wrap=True)

    # Add a legend
    plt.legend()

    # Show the plot
    plt.grid(True)
    plt.tight_layout()

    # Save the plot as an image file
    save_path = os.path.join(save_dir, f'Low_HeatPotential_Potentials_1.2e6TJ_{subsector_name.replace(", ", "_").replace(" ", "_")}_flipped_TJ.png')
    plt.savefig(save_path, bbox_inches='tight')
    plt.show()

# Iterate through each subsector and generate the corresponding plot for low H2 potentials
for subsector, colormap in subsectors_colormap.items():
    # Filter the DataFrame for the current subsector
    df_subsector = df_grundkapital[df_grundkapital['Eurostat_Name'].str.contains(subsector, case=False, na=False)].copy()
    plot_low_potentials_for_subsector(df_subsector, subsector, colormap, threshold=1.2e6)